#  seq2seq + Luong 注意力 进行中英文翻译

In [4]:
import numpy as np
import jieba
from collections import Counter#计数器
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence ,pack_padded_sequence,pad_packed_sequence
from torchtext.data.utils import get_tokenizer#分词器
#from autonotebook import tqdm as notebook_tqdm
import gc

In [3]:
from transformers import AutoTokenizer

In [7]:
UNK_IDX = 0 #未知
PAD_IDX = 1  #
BATCH_SIZE = 64   
EPOCHS  = 30
DROPOUT = 0.2
ENC_HIDDEN_SIZE = DEC_HIDDEN_SIZE = 100
EMBED_SIZE = 100
## 数据集文件
train_en_file = 'en-zh/train.en'
train_cn_file = 'en-zh/train.zh'
test_en_file = 'en-zh/test.en'
test_cn_file = 'en-zh/test.zh'
save_file = 'model.pt'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
#分词器
tokenizer_en = get_tokenizer('basic_english')#按空格进行分割
tokenizer_cn = get_tokenizer(jieba.lcut)#进行结巴分词

In [5]:
#加载文件


def load_data(path,language):
    text = []
    # 语言为中文
    if language == 'cn':
        i = 0
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                i = i+1
                
                if i%1000000 == 0:
                    print(str(i))
                    
                try:
                    line = line.strip().split('\t')
                    text.append(["BOS"] + tokenizer_cn(line[0]) + ["EOS"])
                    
                    
                except Exception as e:
                    print("error raise!")
                    
        
        return text
    
    # 语言为英文
    elif language == 'en':
        i = 0
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                i = i+1
                
                
                try:
                    line = line.strip().split('\t')
                    text.append(["BOS"] + tokenizer_en(line[0].lower()) + ["EOS"])#小写
                    
                    
                except Exception as e:
                    print("error raise!")
                if i%1000000 == 0:
                    print(str(i))
                    
                    
       
        return text
    
    # 语言非中文和非英文
    else:
        print("Can't handle the language!")
        return 
    
   

In [6]:
train_en = load_data(train_en_file, 'en')

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000


In [ ]:
print(len(train_en))

In [9]:

train_zh = load_data(train_cn_file, 'cn')


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.690 seconds.
Prefix dict has been built successfully.


1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000


In [10]:
import pickle
with open('train_zh.pickle', 'wb') as file:
    pickle.dump(train_zh, file)

In [7]:
import pickle
with open('train_en.pickle', 'wb') as file:
    pickle.dump(train_en, file)
#with open('train_zh.pickle', 'wb') as file:
 #   pickle.dump(train_zh, file)

In [3]:
%%time
import pickle
with open('train_en.pickle', 'rb') as file:
    train_en = pickle.load(file)

CPU times: user 2min 8s, sys: 30.8 s, total: 2min 39s
Wall time: 2min 39s


In [4]:
train_en[0], len(train_en)

(['BOS', 'resolution', '918', '(', '1994', ')', 'EOS'], 15886041)

In [ ]:
%%time
with open('train_zh.pickle', 'rb') as file:
    train_zh = pickle.load(file)

In [10]:
test_en = load_data(test_en_file, 'en')


In [12]:
test_en[0]

['BOS',
 '7439th',
 'meeting',
 ',',
 'held',
 'on',
 '11',
 'may',
 '2015',
 '.',
 'EOS']

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
ids = tokenizer.convert_tokens_to_ids(test_en[0])
ids

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

[100, 100, 2309, 117, 1316, 1113, 1429, 1336, 1410, 119, 100]

In [29]:
i = 0
text = []
id = []
with open(test_en_file, 'r', encoding='utf-8') as f:
    for line in f:
        i = i+1

        if i%1000000 == 0:
            print(str(i))

        try:
            line = line.strip().split('\t')
            tokens = tokenizer.tokenize(line[0])
            text.append(tokens)
            id.append(tokenizer.convert_tokens_to_ids(tokens))


        except Exception as e:
            print("error raise!")

In [34]:
with open(test_en_file, 'r', encoding='utf-8') as f:
    tmp = f.read()

In [36]:
tmp[:1000]

"7439th meeting, held on 11 May 2015.\nISIL itself has published videos depicting people being subjected to a range of abhorrent punishments, including stoning, being pushed-off buildings, decapitation and crucifixion.\nUNICEF disbursed emergency cash assistance to tens of thousands of displaced families in camps and UNHCR distributed cash assistance to vulnerable families which had been internally displaced.\n31. Recognizes the important contribution of the African Peer Review Mechanism since its inception in improving governance and supporting socioeconomic development in African countries, and recalls in this regard the high-level panel discussion held on 21 October 2013 on Africa's innovation in governance through 10 years of the African Peer Review Mechanism, organized during the sixty-eighth session of the General Assembly to commemorate the tenth anniversary of the Mechanism;\nSpreads between sovereign bonds in Germany and those in other countries were relatively unaffected by p

In [33]:
text[:10]

[['74',
  '##39',
  '##th',
  'meeting',
  ',',
  'held',
  'on',
  '11',
  'May',
  '2015',
  '.'],
 ['ISIL',
  'itself',
  'has',
  'published',
  'videos',
  'depicting',
  'people',
  'being',
  'subjected',
  'to',
  'a',
  'range',
  'of',
  'a',
  '##b',
  '##hor',
  '##rent',
  'punishment',
  '##s',
  ',',
  'including',
  's',
  '##ton',
  '##ing',
  ',',
  'being',
  'pushed',
  '-',
  'off',
  'buildings',
  ',',
  'de',
  '##cap',
  '##itation',
  'and',
  'c',
  '##ru',
  '##ci',
  '##fi',
  '##xi',
  '##on',
  '.'],
 ['UN',
  '##IC',
  '##EF',
  'di',
  '##s',
  '##bur',
  '##sed',
  'emergency',
  'cash',
  'assistance',
  'to',
  'tens',
  'of',
  'thousands',
  'of',
  'displaced',
  'families',
  'in',
  'camps',
  'and',
  'UN',
  '##HC',
  '##R',
  'distributed',
  'cash',
  'assistance',
  'to',
  'vulnerable',
  'families',
  'which',
  'had',
  'been',
  'internally',
  'displaced',
  '.'],
 ['31',
  '.',
  'Re',
  '##co',
  '##gni',
  '##zes',
  'the',
  'impor

In [28]:
id[:10]

[[5692, 24786, 1582, 2309, 117, 1316, 1113, 1429, 1318, 1410, 119],
 [21388,
  2111,
  1144,
  1502,
  6581,
  10726,
  1234,
  1217,
  13927,
  1106,
  170,
  2079,
  1104,
  170,
  1830,
  13252,
  17759,
  7703,
  1116,
  117,
  1259,
  188,
  1633,
  1158,
  117,
  1217,
  2873,
  118,
  1228,
  2275,
  117,
  1260,
  25265,
  12633,
  1105,
  172,
  5082,
  6617,
  8702,
  8745,
  1320,
  119],
 [7414,
  9741,
  14663,
  4267,
  1116,
  19364,
  5591,
  5241,
  5948,
  5052,
  1106,
  17265,
  1104,
  4674,
  1104,
  13577,
  2073,
  1107,
  7869,
  1105,
  7414,
  15779,
  2069,
  4901,
  5948,
  5052,
  1106,
  8018,
  2073,
  1134,
  1125,
  1151,
  19266,
  13577,
  119],
 [1955,
  119,
  11336,
  2528,
  22152,
  11846,
  1103,
  1696,
  6436,
  1104,
  1103,
  2170,
  153,
  8284,
  4960,
  2508,
  18546,
  1863,
  1290,
  1157,
  12548,
  1107,
  9248,
  12711,
  1105,
  4374,
  20650,
  25726,
  1718,
  1107,
  2170,
  2182,
  117,
  1105,
  19120,
  1107,
  1142,
  7328,


In [41]:
decode = tokenizer.decode(id[1])
decode

'ISIL itself has published videos depicting people being subjected to a range of abhorrent punishments, including stoning, being pushed - off buildings, decapitation and crucifixion.'

'7439th meeting, held on 11 May 2015.'

In [22]:
test_zh = load_data(test_cn_file, 'cn')


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.723 seconds.
Prefix dict has been built successfully.


In [5]:
#构建词汇表
def build_dict(sentences, max_words = 50000):

    vocab = Counter(np.concatenate(sentences)).most_common(max_words)#最大单词数是50000
    word_to_id = {w[0]: index + 2 for index, w in enumerate(vocab)}
    word_to_id['UNK'] = UNK_IDX  #0
    word_to_id['PAD'] = PAD_IDX  #1
    id_to_word = {v: k for k, v in word_to_id.items()}
    return word_to_id,id_to_word
en_wtoi, en_itow = build_dict(train_en)
#zh_wtoi, zh_itow = build_dict(train_zh)

MemoryError: Unable to allocate 647. GiB for an array with shape (452561528,) and data type <U384

# 到此为止

In [ ]:
en_itow

In [ ]:
# 利用词典对原始句子编码 单词->数字
def encode(en_sentences, ch_sentences, en_wtoi, zh_wtoi, sort_by_len=True):


    out_en_sentences = [[en_wtoi.get(w, UNK_IDX) for w in sent] for sent in en_sentences]
    out_ch_sentences = [[zh_wtoi.get(w, UNK_IDX) for w in sent] for sent in ch_sentences]
    #返回w对应的值，否则返回UNK_IDX
    def len_argsort(seq):#按照长度进行排序
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))
       
    # 把中文和英文按照同样的顺序排序
    if sort_by_len:
        sorted_index = len_argsort(out_en_sentences)
        out_en_sentences = [out_en_sentences[i] for i in sorted_index]
        out_ch_sentences = [out_ch_sentences[i] for i in sorted_index]
        
    return out_en_sentences, out_ch_sentences

train_en_encode, train_zh_encode = encode(train_en, train_zh, en_wtoi, zh_wtoi)
#dev_en_encode, dev_zh_encode = encode(dev_en, dev_zh, en_wtoi, zh_wtoi)
test_en_encode, test_zh_encode = encode(test_en, test_zh, en_wtoi, zh_wtoi)


In [ ]:
def encode(sentences, wtoi, sort_by_len=True):


    out_sentences = [[wtoi.get(w, UNK_IDX) for w in sent] for sent in sentences]
    #返回w对应的值，否则返回UNK_IDX
    def len_argsort(seq):#按照长度进行排序
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))
       
    # 把中文和英文按照同样的顺序排序
    if sort_by_len:
        sorted_index = len_argsort(out_en_sentences)
        out_en_sentences = [out_en_sentences[i] for i in sorted_index]
        out_ch_sentences = [out_ch_sentences[i] for i in sorted_index]
        
    return out_en_sentences, out_ch_sentences

train_en_encode, train_zh_encode = encode(train_en, train_zh, en_wtoi, zh_wtoi)
#dev_en_encode, dev_zh_encode = encode(dev_en, dev_zh, en_wtoi, zh_wtoi)
test_en_encode, test_zh_encode = encode(test_en, test_zh, en_wtoi, zh_wtoi)

In [ ]:
test_en_encode

In [ ]:
#返回每个batch的id
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size) 
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx + minibatch_size, n)))
    return minibatches

In [ ]:
get_minibatches(50, 10, shuffle=True)#得到每一个batch对应的id

In [ ]:
#将句子对划分到batch
def get_batches(en_encode, ch_encode):
    batch_indexs = get_minibatches(len(en_encode),BATCH_SIZE)

    batches = []
    for batch_index in batch_indexs:
        batch_en = [torch.tensor(en_encode[index]).long() for index in batch_index] #每一个idx对应的句子，转为tensor格式
        batch_zh = [torch.tensor(ch_encode[index]).long() for index in batch_index]
        length_en = torch.tensor([len(en)  for en in batch_en]).long()#每一个句子的长度
        length_zh = torch.tensor([len(zh)  for zh in batch_zh]).long()

        batch_en = pad_sequence(batch_en, padding_value=PAD_IDX, batch_first=True)#讲一个batch中的句子padding为相同长度
        batch_zh = pad_sequence(batch_zh, padding_value=PAD_IDX, batch_first=True)

        batches.append((batch_en, batch_zh,length_en,length_zh))
    return batches
train_data = get_batches(train_en_encode, train_zh_encode)
dev_data = get_batches(dev_en_encode, dev_zh_encode)

In [ ]:
train_data[0][3]

## 建立模型

![](https://cdn.mathpix.com/snip/images/8Es5WLO-mn8kY7GO-T7o1IxWUBPbZeLrz3JbqY5U2Vw.original.fullsize.png)

其中 $\bar{h}_{s}$ 表示encoder每个hidden_state的输出， $h_{t}$ 表示decoder每个hidden_state的输出。


$$
\begin{aligned}
a_{t}(s) 
&=\frac{\exp \left(\operatorname{score}\left(h_{t}, \bar{h}_{s}\right)\right)}{\sum_{s^{\prime}} \exp \left(\operatorname{score}\left(h_{t}, \bar{h}_{s^{\prime}}\right)\right)} \\
c_{t} &= \sum a_{t} \bar{h}_{s} \\
\tilde{h}_{t} &=\tanh \left(W_{c}\left[c_{t} ; h_{t}\right]\right)\\

\end{aligned}
$$

$\operatorname{score}\left(\boldsymbol{h}_{t}, \overline{\boldsymbol{h}}_{s}\right)= \begin{cases}\boldsymbol{h}_{t}^{\top} \overline{\boldsymbol{h}}_{s} & \text { dot } \\ \boldsymbol{h}_{t}^{\top} \boldsymbol{W}_{\boldsymbol{a}} \overline{\boldsymbol{h}}_{s} & \text { general } \\ \boldsymbol{v}_{a}^{\top} \tanh \left(\boldsymbol{W}_{\boldsymbol{a}}\left[\boldsymbol{h}_{t} ; \overline{\boldsymbol{h}}_{s}\right]\right) & \text { concat }\end{cases}$

In [ ]:
class LuongEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size,enc_hidden_size, dec_hidden_size, dropout):
        super(LuongEncoder,self).__init__()

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.rnn = nn.GRU(embed_size, enc_hidden_size, bidirectional=True)#双向GRU
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_hidden_size*2 , dec_hidden_size)

    def forward(self, x, x_lengths):
        """
        input_seqs : batch_size,max(x_lengths)
        input_lengths: batch_size
        """
        embedded = self.dropout(self.embedding(x))#batch_size,max(x_lengths),embed_size
        packed = pack_padded_sequence(embedded, x_lengths.long().cpu().data.numpy(),batch_first=True,enforce_sorted=False)
        #压缩填充张量,压缩掉无效的填充值
        #enforce_sorted：如果是 True ，则输入应该是按长度降序排序的序列。如果是 False ，会在函数内部进行排序 
        outputs, hidden = self.rnn(packed)
        outputs, _ = pad_packed_sequence(outputs,padding_value =PAD_IDX,batch_first=True)#还原
        
        #hidden (2, batch_size, enc_hidden_size)
        #outputs (batch_size,seq_len, 2 * enc_hidden_size)
        
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)
        hidden = torch.tanh(self.fc(hidden)).unsqueeze(0)
        return outputs, hidden

In [ ]:
class Attn(nn.Module):
    def __init__(self, enc_hidden_size,dec_hidden_size):
        super(Attn,self).__init__()
        #general attention
        self.linear_in = nn.Linear(enc_hidden_size*2, dec_hidden_size, bias=False)
        self.linear_out = nn.Linear(enc_hidden_size*2 + dec_hidden_size, dec_hidden_size)


    def forward(self, output, encoder_out,mask):
        """
        output:batch_size, max(y_lengths), dec_hidden_size  #(h_t)
        encoder_out:batch_size, max(x_lengths), 2 * enc_hidden_size  #(h_s)
        """
        batch_size = output.shape[0]
        output_len = output.shape[1]
        input_len = encoder_out.shape[1]

        encoder_out1 = self.linear_in(encoder_out.view(batch_size*input_len, -1)).view(batch_size, input_len, -1) 
        #Wh_s 
        #batch_size,max(x_lengths),dec_hidden_size
        score = torch.bmm(output, encoder_out1.transpose(1,2))#实现三维数组的乘法，而不用拆成二维数组使用for循环解决
        #[batch_size,max(y_lengths),dec_hidden_size] * [batch_size,dec_hidden_size,max(x_lengths)]
        #batch_size,max(y_lengths),max(x_lengths)#score = h_t W h_s
        score.data.masked_fill(mask,-1e16)
        attn = F.softmax(score, dim=2) #attention系数矩阵

        ct = torch.bmm(attn, encoder_out) #ct = aths
        #[batch_size,max(y_lengths),max(x_lengths)] * [batch_size, max(x_lengths), 2 * enc_hidden_size]
        #batch_size, max(y_lengths), enc_hidden_size*2
        output = torch.cat((ct, output), dim=2) 

        output = output.view(batch_size*output_len, -1)
        output = torch.tanh(self.linear_out(output))
        output = output.view(batch_size, output_len, -1)
        #batch_size, max(y_lengths), dec_hidden_size

        return output,attn


In [ ]:
class LuongDecoder(nn.Module):
    def __init__(self,vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout):
        super(LuongDecoder,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.attention = Attn(enc_hidden_size,dec_hidden_size)
        self.rnn = nn.GRU(embed_size,dec_hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(dec_hidden_size, vocab_size)
    def creat_mask(self,x,y):

        x_mask = x.data != PAD_IDX #batch_size,max(x_lengths)
        y_mask = y.data != PAD_IDX #batch_size,max(y_lengths)
        mask = (1 - (x_mask.unsqueeze(2) * y_mask.unsqueeze(1)).float()).bool()
        #batch_size,max(x_lengths),max(y_lengths)
        return mask 
    
    def forward(self,encoder_out,x,y,y_lengths,hid):
        mask = self.creat_mask(y,x)
        y = self.dropout(self.embedding(y))
        packed = pack_padded_sequence(y, y_lengths.long().cpu().data.numpy(),batch_first=True,enforce_sorted=False)
        out, hid = self.rnn(packed,hid)
       
        out, _ = pad_packed_sequence(out,padding_value =PAD_IDX,batch_first=True)
        
        output,attn = self.attention(out,encoder_out,mask)
        output = self.out(output)
        #batch_size, max(y_lengths), vocab_size
        return output,hid,attn

In [ ]:
class seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(seq2seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self,x,x_lengths,y,y_lengths):
        encoder_out,hid = self.encoder(x,x_lengths)
        output,hid,attn = self.decoder(encoder_out,  #这里输出的hid是decoder_rnn的hid
                    x=x,
                    y=y,
                    y_lengths=y_lengths,
                    hid=hid)#encoder的hid
        return output,attn

    def translate(self, x, x_lengths, y, max_length=15):
        encoder_out, hid = self.encoder(x, x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for _ in range(max_length):
            output,hid,attn = self.decoder(encoder_out, 
                    x=x,
                    y=y,
                    y_lengths=torch.ones(batch_size).long().to(y.device),
                    hid=hid)

            y = output.max(2)[1].view(batch_size, 1)

            preds.append(y)
            attns.append(attn)
        return torch.cat(preds, 1),torch.cat(attns, 1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Define Model
encoder = LuongEncoder(vocab_size = len(en_itow), embed_size = EMBED_SIZE, enc_hidden_size = ENC_HIDDEN_SIZE, dec_hidden_size = DEC_HIDDEN_SIZE, dropout = DROPOUT)
decoder = LuongDecoder(vocab_size = len(zh_itow), embed_size = EMBED_SIZE, enc_hidden_size  = ENC_HIDDEN_SIZE, dec_hidden_size = DEC_HIDDEN_SIZE, dropout = DROPOUT)
model = seq2seq(encoder, decoder)
model = model.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)#忽略padding位置的损失
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train_epoch(model, optimizer,train_data):
    model.train()
    losses = 0
    for x, y, x_lengths, y_lengths in train_data:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        x_lengths = x_lengths.to(DEVICE)

        y_input = y[:,:-1]#将前seq-1个单词作为输入
        y_output = y[:,1:]#将后seq-1个单词作为输出，相当于前一个单词预测后一个单词
        y_lengths = (y_lengths-1).to(DEVICE)


        logits, _ = model(x,x_lengths,y_input,y_lengths)#batch_size, max(y_lengths), vocab_size
        loss = loss_fn(logits.reshape(-1,logits.shape[-1]), y_output.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()

    return losses / len(train_data)


def evaluate(model,dev_data):

    model.train()
    losses = 0
    for x, y, x_lengths, y_lengths in train_data:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        x_lengths = x_lengths.to(DEVICE)

        y_input = y[:,:-1]
        y_output = y[:,1:]
        y_lengths = (y_lengths-1).to(DEVICE)
        logits, _ = model(x,x_lengths,y_input,y_lengths)
        loss = loss_fn(logits.reshape(-1,logits.shape[-1]), y_output.reshape(-1))

        losses += loss.item()

    return losses / len(train_data)

In [ ]:
from timeit import default_timer as timer


for epoch in range(1, EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(model, optimizer,train_data)
    end_time = timer()
    val_loss = evaluate(model,dev_data)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

In [ ]:
def translate_dev(i):
    model.eval()
    
    en_sent = " ".join([en_itow[word] for word in test_en_encode[i]])
    print('英文原句：',en_sent)
    print('标准中文翻译：'," ".join([zh_itow[word] for word in test_zh_encode[i]]))

    bos = torch.Tensor([[zh_wtoi["BOS"]]]).long().to(DEVICE)
    x = torch.Tensor(test_en_encode[i]).long().to(DEVICE).reshape(1, -1)
    x_len = torch.Tensor([len(test_en_encode[i])]).long().to(DEVICE)
    
    translation,_ = model.translate(x, x_len, bos)
    translation = [zh_itow[i] for i in translation.data.cpu().numpy().reshape(-1)]

    trans = []
    for word in translation:
        if word != "EOS":
            trans.append(word)
        else:
            break
    print('模型翻译结果：'," ".join(trans))
for i in range(50,100):
    translate_dev(i)
    print()

# 练习一：Bi-LSTM + attention 用于情感分类

补全代码：我们使用构建一个Bi-LSTM + attention模型完成文本分类任务，数据使用IMDb电影评论数据集，检测一段文字的情感是正面还是负面。

[论文](https://aclanthology.org/P16-2034.pdf)

![](https://cdn.mathpix.com/snip/images/pvB4-X5G9OAFQ_A2wYqjCZxoOOMu_u1PpkkrJUlTbQ8.original.fullsize.png)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence 

from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

## 准备数据

In [ ]:
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
vocab.insert_token("<pad>",1)


In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x : 0 if x == 'neg' else 1

In [ ]:
def collate_batch(batch):#自定义的batch输出
    label_list, text_list,lengths = [], [],[]
    batch.sort(key=lambda x: len(text_pipeline(x[1])), reverse=True)#按照长度的大小进行排序
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text))
        text_list.append(processed_text)
        lengths.append(len(processed_text))
    text_list = pad_sequence(text_list, padding_value=vocab.get_stoi()["<pad>"], batch_first=True)#进行填充，每个batch中的句子需要有相同的长度
    return torch.tensor(label_list), text_list, lengths

In [ ]:
train_iter,test_iter = IMDB(root='data', split=('train', 'test'))
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ =  random_split(train_dataset, [num_train, len(train_dataset) - num_train])


train_dataloader = DataLoader(split_train_, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)

In [ ]:
data = next(iter(train_dataloader))

In [ ]:
data[0]#标签

In [ ]:
data[1]#batch_size,max_length

In [ ]:
data[2]#lengths

## 模型

模型分为五个部分

![](https://cdn.mathpix.com/snip/images/pvB4-X5G9OAFQ_A2wYqjCZxoOOMu_u1PpkkrJUlTbQ8.original.fullsize.png)

- 输入层（Input layer）：将句子输入模型
- 嵌入层（Embedding layer）：将每个词映射到一个低维向量
- LSTM层（LSTM layer）：利用BiLSTM从词向量中获得特征
- Attention层（Attention layer）：生成权重向量，将每个时间步长的单词级特征与权重向量相乘，合并成句子级特征向量（补全代码）
- 输出层（Output layer）： 对句子进行分类

In [ ]:
class bilstm_attn(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 dropout_rate, pad_id):
        super(bilstm_attn,self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_id)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = True,
                            dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        """
            Write your code here.
        """
        
    def forward(self, x,lengths):
        
        embedded = self.dropout(self.embedding(x))# [batch size,seq len] -> [batch size,seq len,embedding_dim]

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # hidden = [n layers *2, batch size, hidden dim]最后一个step的hidden
        # cell = [n layers * 2, batch size, hidden dim]最终一个step的cell
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        # output = [batch size, seq len, hidden dim * 2]#每一个step下的最后一层的output
        output = output.reshape(output.shape[0],output.shape[1],2,-1)
        # output = [batch size, seq len, 2,hidden dim]
        output =  torch.sum(output, dim=2)
        # output = [batch size, seq len, hidden dim]
        output = self.dropout(output)
        
        """
            Write your code here.
        """
        return prediction

In [ ]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 2
n_layers = 1
dropout_rate = 0.5
pad_id = vocab.get_stoi()["<pad>"]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = bilstm_attn(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,dropout_rate, pad_id).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(model, train_loader, optimizer, loss_fn):
    
    
    epoch_loss = 0
    corrects = 0
    total_len = 0
    model.train() #model.train()代表了训练模式    
    for label, text, lengths in train_loader: 
        label = label.to(device)
        text = text.to(device)  
        
        out = model(text,lengths)
        loss = loss_fn(out,label)
        
        _,pred = torch.max(out.data,1)       
        corrects += (pred == label).sum().item()
        
        optimizer.zero_grad() #加这步防止梯度叠加
        loss.backward() #反向传播
        optimizer.step() #梯度下降
        
        epoch_loss += loss.item() * len(label)
        #loss.item()已经本身除以了len(batch.label)
        #所以得再乘一次，得到一个batch的损失，累加得到所有样本损失。

        
        total_len += len(label)
        #计算train_iterator所有样本的数量，不出意外应该是17500
        
    return epoch_loss / total_len, corrects / total_len

In [ ]:
def evaluate(model, valid_loader):
     
    
    epoch_loss = 0
    corrects = 0
    total_len = 0
    
    model.eval()
    #转换成测试模式，冻结dropout层或其他层。
    
    with torch.no_grad():
        for label, text, lengths in valid_loader: 
            #iterator为valid_iterator
            label = label.to(device)
            text = text.to(device) 
            
            out = model(text,lengths)
            loss = loss_fn(out,label)

            _,pred = torch.max(out.data,1)       
            corrects += (pred == label).sum().item()
            
            
            epoch_loss += loss.item() * len(label)
            total_len += len(label)
    model.train() #调回训练模式   
    
    return epoch_loss / total_len, corrects / total_len

In [ ]:
for epoch in range(10):
    
    train_loss, train_acc = train(model, train_dataloader, optimizer, loss_fn)
    print("epoch:",epoch,"train_loss:",train_loss,"train_acc",train_acc)
    valid_loss, valid_acc = evaluate(model, valid_dataloader)  
    print("epoch:",epoch,"valid_loss:",valid_loss,"valid_acc",valid_acc)


In [ ]:
torch.save(model.state_dict(), 'bilstm_attn.pt')

In [ ]:
def predict_sentiment(text):
    text = text_pipeline(text)
    
    length = torch.LongTensor([len(text)])
    tensor = torch.LongTensor(text).unsqueeze(0).to(device)
    
    out = model(tensor, length)
    _,pred = torch.max(out.data,1)
    return pred.item()

In [ ]:
predict_sentiment("This film is terrible")

In [ ]:
predict_sentiment("This film is great")